In [2]:
from datetime import datetime
import yaml
from backbone.utils.general_purpose import load_function
from backbone.utils.wfo_utils import optimization_function, run_wfo
from backbone.utils.wfo_utils import run_strategy
import MetaTrader5 as mt5

if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

root = './backbone/data'

with open('configs/live_trading.yml', 'r') as file:
    strategies = yaml.safe_load(file)

with open('configs/test_creds.yml', 'r') as file:
    creds = yaml.safe_load(file)

date_from = datetime(2021, 9, 8)
date_to = datetime(2024, 11, 10)

run = [
    'backbone.short_ibs.ShortIBS',
    'backbone.triple_supertrend.TripleSuperTrend',    
    'backbone.channel_strategy.Channel',
    'backbone.mean_reversion_strategy.MeanReversion',
]

equity_curves = {}
trades = {}
INITIAL_CASH = 10_000
MARGIN = 1/30

bot_path = 'backbone.trader_bot.TraderBot'

for strategy_name, configs in strategies.items():
    
    print(strategy_name)
    
    if strategy_name not in run:
        continue
    
    instruments_info = configs['instruments_info']
    wfo_params = configs['wfo_params']
    opt_params = configs['opt_params']
    name = configs['name']
    
    for ticker, info in instruments_info.items():
        symbol_info = mt5.symbol_info_tick(ticker)
        avg_price = (symbol_info.bid + symbol_info.ask) / 2
        spread = symbol_info.ask - symbol_info.bid
        commission = round(spread / avg_price, 5)

        cron = info['cron']
        timeframe = info['timeframe']
        
        print(strategy_name, ticker, timeframe)

        strategy = load_function(strategy_name)
        
        bot = load_function(bot_path)(name, ticker, timeframe, creds, opt_params, wfo_params, strategy)
    
        df = bot.get_data(date_from, date_to)
        
        configs = strategies[strategy_name]

        instruments_info = configs['instruments_info']
        wfo_params = configs['wfo_params']
        opt_params = configs['opt_params']

        optimized_params = None

        if wfo_params:
            _, _, optimized_params = run_wfo(
                strategy=bot.strategy,
                ticker=ticker,
                interval=timeframe,
                prices=df,
                initial_cash=INITIAL_CASH,
                commission=commission,
                margin=MARGIN,
                optim_func=optimization_function,
                params=bot.opt_params,
                lookback_bars=bot.wfo_params['look_back_bars'],
                warmup_bars=bot.wfo_params['warmup_bars'],
                validation_bars=250,
                plot=False
            )
        
        df_stats, stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=timeframe,
            commission=commission, 
            prices=df, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=optimized_params, 
            plot=False
        )

        equity_curves[bot.name] = stats['_equity_curve']
        trades[bot.name] = stats['_trades']

backbone.short_ibs.ShortIBS
backbone.short_ibs.ShortIBS ADAUSD H4
  login=1520531177
  trade_mode=0
  leverage=30
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=9999.97
  credit=0.0
  profit=0.0
  equity=9999.97
  margin=0.0
  margin_free=9999.97
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=50.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=FTMO Free Trial Swing USD
  server=FTMO-Demo2
  currency=USD
  company=FTMO S.R.O.
backbone.short_ibs.ShortIBS DOTUSD H4
  login=1520531177
  trade_mode=0
  leverage=30
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=9999.97
  credit=0.0
  profit=0.0
  equity=9999.97
  margin=0.0
  margin_free=9999.97
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=50.0
  margin_initial=0.0
  m

In [3]:
equity_curves.keys()

dict_keys(['ShortIBS_ADAUSD_H4', 'ShortIBS_DOTUSD_H4', 'TripleST_TSLA_H3', 'TripleST_NVDA_H4', 'MeanRev_XMRUSD_H1', 'Channel_BTCUSD_H2', 'Channel_NVDA_H1'])

In [4]:
import pandas as pd

min_date = None
max_date = None

for name, curve in equity_curves.items():
    # Convertir las fechas a UTC si son tz-naive
    actual_date = curve.index[0].tz_localize('UTC') if curve.index[0].tz is None else curve.index[0].tz_convert('UTC')
    
    # Si min_date es None, inicializar con la primera fecha
    if min_date is None:
        min_date = actual_date
    # Comparar si la fecha actual es menor que min_date
    elif actual_date < min_date:
        min_date = actual_date

    # Si max_date es None, inicializar con la última fecha
    curve_last_date = curve.index[-1].tz_localize('UTC') if curve.index[-1].tz is None else curve.index[-1].tz_convert('UTC')
    
    if max_date is None:
        max_date = curve_last_date
    # Comparar si la fecha actual es mayor que max_date
    elif curve_last_date > max_date:
        max_date = curve_last_date

# Mostrar las fechas encontradas
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

# Calcular min_date y max_date
min_date = min_date.date()
max_date = max_date.date()

print(min_date)
print(max_date)

date_range = pd.to_datetime(pd.date_range(start=min_date, end=max_date, freq='D'))
print(date_range)

Min Date: 2021-09-08 03:00:00+00:00
Max Date: 2024-11-10 03:00:00+00:00
2021-09-08
2024-11-10
DatetimeIndex(['2021-09-08', '2021-09-09', '2021-09-10', '2021-09-11',
               '2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15',
               '2021-09-16', '2021-09-17',
               ...
               '2024-11-01', '2024-11-02', '2024-11-03', '2024-11-04',
               '2024-11-05', '2024-11-06', '2024-11-07', '2024-11-08',
               '2024-11-09', '2024-11-10'],
              dtype='datetime64[ns]', length=1160, freq='D')


In [5]:
percentual_differences = {}

def get_percentual_differences(equity_curves, trades):
    for name, curve in equity_curves.items():
        # trade_df = trades[name]
        equity_df = equity_curves[name]
        
        
        # trade_df['ExitTime'] = trade_df['ExitTime'].dt.floor('D')
        
        # trade_df = pd.DataFrame(trade_df.groupby('ExitTime')['PnL'].sum()).reindex(date_range)
        # trade_df.fillna(0, inplace=True)
        
        equity_df = equity_df.reset_index().rename(columns={'index':'Date'})
        equity_df['Date'] = pd.to_datetime(equity_df['Date'])
        equity_df['Date'] = equity_df['Date'].dt.floor('D')
        
        equity_df = pd.DataFrame(equity_df.groupby('Date')['Equity'].last()).reindex(date_range)
        equity_df.ffill(inplace=True)
        equity_df.fillna(INITIAL_CASH, inplace=True)
  
        # pct_df = pd.merge(
        #     trade_df,
        #     equity_df,
        #     left_index=True,
        #     right_index=True
        # )

        equity_df['diff'] = equity_df['Equity'] - equity_df['Equity'].shift(1)
        
        percentual_differences[name] = equity_df
    
    return percentual_differences


differences = get_percentual_differences(equity_curves, trades)
differences

{'ShortIBS_ADAUSD_H4':                   Equity        diff
 2021-09-08  10000.000000         NaN
 2021-09-09  10000.000000    0.000000
 2021-09-10  10000.000000    0.000000
 2021-09-11  10000.000000    0.000000
 2021-09-12  10000.000000    0.000000
 ...                  ...         ...
 2024-11-06   8305.276987 -148.358452
 2024-11-07   8305.276987    0.000000
 2024-11-08   8305.276987    0.000000
 2024-11-09   8305.276987    0.000000
 2024-11-10   8305.276987    0.000000
 
 [1160 rows x 2 columns],
 'ShortIBS_DOTUSD_H4':                   Equity       diff
 2021-09-08  10000.000000        NaN
 2021-09-09  10000.000000   0.000000
 2021-09-10  10000.000000   0.000000
 2021-09-11  10000.000000   0.000000
 2021-09-12  10000.000000   0.000000
 ...                  ...        ...
 2024-11-06  12215.200681  -7.175112
 2024-11-07  12122.100681 -93.100000
 2024-11-08  12091.078506 -31.022175
 2024-11-09  12091.078506   0.000000
 2024-11-10  12091.078506   0.000000
 
 [1160 rows x 2 columns],


In [6]:

def get_hipotetical_wallet_equity(equity_curves, initial_equity):
    total = pd.DataFrame()
    variaciones_porcentuales = {}

    for name, curve in equity_curves.items():

        eq = equity_curves[name].copy()
        eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')
        eq['Date'] = pd.to_datetime(eq['Date'])
        eq['Date'] = eq['Date'].dt.floor('D')

        eq = eq.groupby('Date').agg({'Equity':'last'})

        eq = eq.reindex(date_range)
        
        eq.Equity = eq.Equity.ffill()
        eq.Equity = eq.Equity.fillna(INITIAL_CASH)
    
        eq['variacion'] = eq['Equity'] - eq['Equity'].shift(1)
        eq['variacion_porcentual'] = eq['variacion'] / eq['Equity'].shift(1)
        
        df_variacion = pd.DataFrame(
            {
                f'variacion_{name}': eq.variacion_porcentual.fillna(0)
            }
        )
        
        total = pd.concat([total, df_variacion], axis=1)

    total = total.reset_index().rename(columns={'index':'Date'})

    # Inicializa el valor de equity
    total['Equity'] = initial_equity

    # Lista de columnas con las variaciones porcentuales
    variation_cols = [col for col in total.columns if col.startswith('variacion')]

    # Calcular la curva de equity
    for i in range(1, len(total)):
        previous_equity = total.loc[i-1, 'Equity']  # Equity del periodo anterior
        
        # Calcula el impacto monetario de cada bot por separado y suma el resultado
        impact_sum = 0
        for col in variation_cols:
            variation = total.loc[i, col]
            impact_sum += previous_equity * variation
        
        # Actualiza el equity sumando el impacto monetario total
        total.loc[i, 'Equity'] = previous_equity + impact_sum

    # Resultado final

    total = total.set_index('Date')
    return total[['Equity']]

In [25]:
if 'variaciones_porcentuales' in equity_curves.keys():
    del equity_curves['variaciones_porcentuales']

initial_equity = 10000
total = get_hipotetical_wallet_equity(equity_curves=equity_curves, initial_equity=initial_equity)
equity_curves['variaciones_porcentuales'] = total


In [26]:
equity_curves.keys()

dict_keys(['TripleST_TSLA_H3', 'Channel_BTCUSD_H2', 'Channel_NVDA_H1', 'variaciones_porcentuales'])

In [27]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
for k, v in equity_curves.items():
    
    fig.add_trace(go.Scatter(x=v.index, y=v.Equity, mode='lines', name=k))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curvas de Equity de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()


In [28]:
# ShortIBS_ADAUSD_H4 = equity_curves['ShortIBS_ADAUSD_H4']
# ShortIBS_DOTUSD_H4 = equity_curves['ShortIBS_DOTUSD_H4']
# TripleST_NVDA_H4 = equity_curves['TripleST_NVDA_H4']
# MeanRev_XMRUSD_H4 = equity_curves['MeanRev_XMRUSD_H1']

# del equity_curves['ShortIBS_ADAUSD_H4']
# del equity_curves['ShortIBS_DOTUSD_H4']
# del equity_curves['TripleST_NVDA_H4']
# del equity_curves['MeanRev_XMRUSD_H1']


# equity_curves['ShortIBS_ADAUSD_H4'] = ShortIBS_ADAUSD_H4
# equity_curves['ShortIBS_DOTUSD_H4'] = ShortIBS_DOTUSD_H4
# equity_curves['TripleST_NVDA_H4'] = TripleST_NVDA_H4
# equity_curves['MeanRev_XMRUSD_H4'] = MeanRev_XMRUSD_H4

In [29]:
import numpy as np

def max_drawdown(equity_curve):
    # Calcular el running max de la equity curve
    running_max = np.maximum.accumulate(equity_curve)
    
    # Calcular el drawdown
    drawdown = (equity_curve - running_max) / running_max
    
    # Encontrar el valor máximo de drawdown y la fecha correspondiente
    max_drawdown_value = np.min(drawdown) * 100  # Convertir el drawdown a porcentaje
    max_drawdown_date = equity_curve.index[np.argmin(drawdown)]
    
    print(f"Máximo drawdown: {max_drawdown_value:.2f}%")
    print(f"Fecha del máximo drawdown: {max_drawdown_date}")



for strategy, equity in equity_curves.items():
    print(strategy)
    max_drawdown(equity.loc['2021-01-01 08:00:00':].Equity)
    print('=================================')

TripleST_TSLA_H3
Máximo drawdown: -9.92%
Fecha del máximo drawdown: 2022-09-30 15:00:00
Channel_BTCUSD_H2
Máximo drawdown: -18.00%
Fecha del máximo drawdown: 2023-06-05 02:00:00
Channel_NVDA_H1
Máximo drawdown: -13.59%
Fecha del máximo drawdown: 2022-06-24 16:00:00
variaciones_porcentuales
Máximo drawdown: -14.76%
Fecha del máximo drawdown: 2022-03-16 00:00:00


In [30]:
import pandas as pd
import numpy as np
import plotly.express as px

# Supongamos que tienes el DataFrame `df` como en el ejemplo
# Convertir el índice a tipo datetime si no lo está
df = equity_curves["variaciones_porcentuales"]
df.index = pd.to_datetime(df.index)

# Calcular los retornos diarios en porcentaje
df['Daily Return'] = ((df['Equity'] - df['Equity'].iloc[0]) / df['Equity'].iloc[0]) * 100

# Crear un DataFrame resampleado con valores mínimo y máximo para cada mes
monthly_min = df['Daily Return'].resample("M").min()
monthly_max = df['Daily Return'].resample("M").max()

# Aplicar la lógica de np.where para seleccionar el mínimo si < 0, máximo si >= 0
monthly_returns = np.where(monthly_max >= 0, monthly_max, monthly_min)

# Crear un índice temporal basado en las fechas de los datos mensuales
monthly_index = df['Daily Return'].resample("M").apply(lambda x: x.index[-1])

# Crear un DataFrame para el gráfico
monthly_df = pd.DataFrame({
    'Fecha': monthly_index,
    'Retorno Mensual': monthly_returns
})

# Crear el gráfico de barras con Plotly
fig = px.bar(
    monthly_df,
    x='Fecha',
    y='Retorno Mensual',
    labels={"Fecha": "Fecha", "Retorno Mensual": "Retorno Mensual (%)"},
    title="Retornos Mensuales Ajustados en Porcentaje",
    text_auto='.2s'
)

fig.update_layout(
    # width=800,
    # height=700,
    xaxis_title="Fecha",
    yaxis_title="Retorno Mensual (%)",
    xaxis=dict(tickformat="%Y-%m"),
    bargap=0,  # Reduce el espacio entre barras (0 = sin espacio, 1 = barras separadas completamente)
)

fig.show()


In [35]:
monthly_df['diferencia_porcentual'] = monthly_df['Retorno Mensual'] - monthly_df['Retorno Mensual'].shift(1)

display(monthly_df)

print('Diferencia porcentual < -10: ', monthly_df[monthly_df['diferencia_porcentual'] < -10].shape[0])
print('Diferencia porcentual > 10: ', monthly_df[monthly_df['diferencia_porcentual'] > 10].shape[0])

,Fecha,Retorno Mensual,diferencia_porcentual
Date,,,
2021-09-30,2021-09-30,1.152881,NaN
2021-10-31,2021-10-31,39.050050,37.897169
2021-11-30,2021-11-30,61.100301,22.050251
2021-12-31,2021-12-31,51.748852,-9.351448
2022-01-31,2022-01-31,60.341519,8.592667
2022-02-28,2022-02-28,56.746127,-3.595393
2022-03-31,2022-03-31,56.659445,-0.086682
2022-04-30,2022-04-30,56.770960,0.111515
2022-05-31,2022-05-31,66.824004,10.053044


Diferencia porcentual < -10:  2
Diferencia porcentual > 10:  18


In [14]:
del equity_curves['variaciones_porcentuales']

# Correlaciones

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('M').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('W').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()

In [ ]:
correlation_matrix

In [ ]:
# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(all_equity_df.index, all_equity_df['profit'], color='skyblue', width=10)
plt.xlabel('Fecha')
plt.ylabel('Suma de la fila')
plt.title('Suma de cada fila a lo largo del tiempo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
non_corr = correlation_matrix[correlation_matrix<0.2]
non_corr


In [ ]:
(equity_curves['variaciones_porcentuales'] - equity_curves['variaciones_porcentuales'].shift(1)).std()

In [ ]:
all_equity_df.describe()

In [ ]:
summary = all_equity_df.describe()
mean_std = summary.loc['std'].mean()
mean_std

In [ ]:
summary.loc['std'].mean() / summary.loc['mean'].mean()

In [ ]:
(equity_curves['variaciones_porcentuales'] - equity_curves['variaciones_porcentuales'].shift(1)).std() / (equity_curves['variaciones_porcentuales'] - equity_curves['variaciones_porcentuales'].shift(1)).mean()

In [ ]:
uncorrelation_matrix = correlation_matrix[correlation_matrix < 0.6]
uncorrelation_matrix

In [ ]:
highcorrelation_matrix = correlation_matrix[correlation_matrix > 0.6]
highcorrelation_matrix

In [ ]:
a = [0.02, -0.01, 0.05, 0.03, -0.02]
b = [0.01, -0.02, 0.04, 0.01, -0.01]

pd.DataFrame({'a':a, 'b':b,}).corr()